# Guliford_model_using LSTM

In [10]:
#Import all the necessary libraries
import os
import time
import re

import numpy as np
import pandas as pd

from bs4 import BeautifulSoup

import matplotlib.pyplot as plt
import seaborn as sns 

from io import StringIO

import nltk
from nltk.corpus import stopwords 

from keras.models import load_model
from keras.models import model_from_json
#To ignore warning 
import warnings
warnings.filterwarnings("ignore")

In [11]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense, Embedding

from keras.utils.np_utils import to_categorical
from keras.layers import Activation, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [12]:
from nltk import ConfusionMatrix
from sklearn.metrics import accuracy_score,classification_report,precision_score,recall_score,precision_recall_curve,roc_curve,confusion_matrix

In [13]:
import h2o
#h2o.init()
#from h2o.estimators.deeplearning import H2ODeepLearningEstimator
#print(h2o.cluster_info())
#print(h2o.ls())


In [14]:
os.chdir("D:/VINODKUMAR/NLP CLASSIFICATION PROJECT/SHASTER-2/SHASTER")

In [15]:
actual_data=pd.read_csv("new_Guliford_data.csv",header=0)

In [16]:
test_data=pd.read_csv("new_Guliford_march_data.csv",header=0)

In [17]:
print(actual_data.shape)
print(test_data.shape)
print(actual_data.columns)
print(test_data.columns)

(86554, 10)
(2279, 10)
Index(['Unnamed: 0', 'FileID', 'clientid', 'PID', 'SUMMARY', 'DATA',
       'Categories1', 'Sub_categories1', 'Previous_Appointment', 'pre_data'],
      dtype='object')
Index(['Unnamed: 0', 'FileID', 'clientid', 'PID', 'SUMMARY', 'DATA',
       'Categories1', 'Sub_categories1', 'Previous_Appointment', 'pre_data'],
      dtype='object')


In [18]:
actual_data_for_train=actual_data[['SUMMARY', 'pre_data','Categories1']]

In [19]:
test_data_for_train=test_data[['SUMMARY', 'pre_data','Categories1']]

In [20]:
# Changing the column names
col_names=["summary","data","categories"]

In [21]:
actual_data_for_train.columns=col_names
test_data_for_train.columns=col_names

In [22]:
actual_data1=actual_data_for_train
test_data1=test_data_for_train

In [23]:
#Converting to lower case
actual_data1['summary'] = actual_data1['summary'].astype(str).str.lower()
actual_data1["data"] = actual_data1["data"].astype(str).str.lower()
actual_data1["categories"] = actual_data1["categories"].map(lambda x: x.upper())

In [24]:
#Converting to lower case
test_data1['summary'] = test_data1['summary'].astype(str).str.lower()
test_data1["data"] = test_data1["data"].astype(str).str.lower()
test_data1["categories"] = test_data1["categories"].map(lambda x: x.upper())

In [25]:
print(actual_data1.dtypes)
print(test_data1.dtypes)

summary       object
data          object
categories    object
dtype: object
summary       object
data          object
categories    object
dtype: object


In [26]:
#Converting datatype of summary feature
actual_data1.summary =actual_data1.summary.astype('str')
#Converting datatype of data feature
actual_data1.data =actual_data1.data.astype('str')
#Converting datatype of category feature
actual_data1.categories=actual_data1.categories.astype("category")

In [27]:
#Converting datatype of summary feature
test_data1.summary =test_data1.summary.astype('str')
#Converting datatype of data feature
test_data1.data =test_data1.data.astype('str')
#Converting datatype of category feature
test_data1.categories=test_data1.categories.astype("category")

In [28]:
#Categoreies coulumn levels count 
actual_data1.categories.value_counts()

ASK_A_DOCTOR     31337
PRESCRIPTION     21490
APPOINTMENTS     19477
MISCELLANEOUS    10848
LAB               3402
Name: categories, dtype: int64

In [29]:
#Categoreies coulumn levels count 
test_data1.categories.value_counts()

ASK_A_DOCTOR     973
PRESCRIPTION     505
APPOINTMENTS     416
MISCELLANEOUS    307
LAB               78
Name: categories, dtype: int64

In [30]:
#Removing Punctuation
actual_data1['data'] = actual_data1['data'].str.replace('[^\w\s\d\*\\b]','')
actual_data1['summary'] = actual_data1['summary'].str.replace('[^\w\s\d\*\\b]','')  

In [31]:
#Removing Punctuation
test_data1['data'] = test_data1['data'].str.replace('[^\w\s\d\*\\b]','')
test_data1['summary'] = test_data1['summary'].str.replace('[^\w\s\d\*\\b]','')

In [32]:
#Replacing spelling errors 
import re
def replacing_text(input_text):
    letters_only = re.sub("[^a-zA-Z]", " ",input_text )
    low_case = letters_only.lower()
    input_text   =re.sub('pt ', ' patient ', low_case)
    input_text1  =re.sub(' appt ', ' appointment ',  input_text)
    input_text1  =re.sub(' appt ', ' appointment ',  input_text)
    input_text2  =re.sub(' ov ', ' office visit ',  input_text1)
    input_text3  =re.sub(' schd ', ' scheduled ',  input_text2)
    input_text4  =re.sub(' np ', ' new patient ',  input_text3)
    input_text5  =re.sub(' r/s ', ' reschedule ',  input_text4)
    input_text6  =re.sub(' med ', ' medication ',  input_text5)
    input_text7  =re.sub(' pa ', ' prior authorization ',  input_text6)
    input_text8  =re.sub(' fin ', ' financial ',  input_text7)
    input_text9  =re.sub(' dept ', ' department ',  input_text8) 
    input_text10 =re.sub(' Call patient back ', ' ', input_text9)
    input_text11 =re.sub(' Phone Note ', ' ', input_text10)
    input_text12 =re.sub(' caller name ', ' ', input_text11)
    input_text13 =re.sub(' call Taken by ', ' ', input_text12)
    input_text14 =re.sub(' cell Phone ', ' ', input_text13)
    input_text15 =re.sub(' call from patient  ', ' ', input_text14)
    input_text16 =re.sub(' call patient back at ', ' ', input_text15)
    input_text17 =re.sub(' additional follow-up details ', ' ', input_text16)
    input_text18 =re.sub(' follow-up details ', ' ', input_text17)
    input_text19 =re.sub(' additional follow-up by ', ' ', input_text18)
    input_text20 =re.sub(' follow-up by ', ' ', input_text19)
    input_text21 =re.sub(' default paragraph ', ' ', input_text20)
    input_text22 =re.sub(' default paragraph ', ' ', input_text21)
    input_text23 =re.sub(' pm ', '  ', input_text22)
    input_text24 =re.sub(' tabs ', ' tablets ', input_text23)
    input_text25 =re.sub(' ph ',' phone ', input_text24)
    input_text26 =re.sub(' dr ',' doctor ',input_text25)
    input_text27 =re.sub(' msg ',' message ', input_text26)
    input_text28 =re.sub(' sched ',' scheduled ', input_text27)
    input_text29 =re.sub(' yrly ',' yearly ', input_text28)
    input_text30 =re.sub('xxx ',' ',input_text29)
    input_text31 =re.sub(' clld ', ' called ', input_text30)
    input_text32 =re.sub(' rs ',' reschedule ',input_text31)
    input_text33 =re.sub(' appt', ' appointment ',  input_text32)
    input_text34 =re.sub(' rn ', ' relay neurology ', input_text33)
    input_text35 =re.sub(' ad ', ' add ', input_text34)
    input_text36 =re.sub(' new medications ', '  ', input_text35)
    input_text37 =re.sub(' summary of call ', ' ', input_text36)
    input_text38 =re.sub('summary of call ',' ',input_text37)
    input_text39 =re.sub(' am ', ' ', input_text38)
    input_text40 =re.sub(' call initial ', ' ', input_text39)
    input_text41 =re.sub(' initial taken ', ' ', input_text40)
    input_text42 =re.sub(' amanda gray',' ', input_text41)
    input_text43 =re.sub(' wife melissa dykes ',' ', input_text42)
    input_text44 =re.sub(' wife relay neurology ',' ', input_text43)
    input_text45 =re.sub(' incoming back ',' ', input_text44)
    input_text46 =re.sub(' kelly sykes',' ', input_text45)
    return(input_text46) 

In [33]:
length_data=len(actual_data1)

In [34]:
start = time.time()
data_process2=[]
for i in range (0,length_data):
    if i%10000 == 0:
        print ("no.of rows reviews processed:",i)
    data_process2.append(replacing_text(actual_data1.data[i]))
    
print("Training took {:0.3f} seconds".format(time.time()-start))

no.of rows reviews processed: 0
no.of rows reviews processed: 10000
no.of rows reviews processed: 20000
no.of rows reviews processed: 30000
no.of rows reviews processed: 40000
no.of rows reviews processed: 50000
no.of rows reviews processed: 60000
no.of rows reviews processed: 70000
no.of rows reviews processed: 80000
Training took 15.297 seconds


In [35]:
start = time.time()
summary_process2=[]
for i in range (0,length_data):
    if i%10000 == 0:
        print ("no.of rows reviews processed:",i)
    summary_process2.append(replacing_text(actual_data1.summary[i]))
print("Training took {:0.3f} seconds".format(time.time()-start))

no.of rows reviews processed: 0
no.of rows reviews processed: 10000
no.of rows reviews processed: 20000
no.of rows reviews processed: 30000
no.of rows reviews processed: 40000
no.of rows reviews processed: 50000
no.of rows reviews processed: 60000
no.of rows reviews processed: 70000
no.of rows reviews processed: 80000
Training took 5.725 seconds


In [36]:
actual_data1.data=pd.DataFrame(data_process2)
actual_data1.summary=pd.DataFrame(summary_process2)

In [37]:
length_test_data=len(test_data1)

In [38]:
start = time.time()
test_data_process2=[]
for i in range (0,length_test_data):
    if i%10000 == 0:
        print ("no.of rows reviews processed:",i)
    test_data_process2.append(replacing_text(test_data1.data[i]))
    
print("Training took {:0.3f} seconds".format(time.time()-start))

no.of rows reviews processed: 0
Training took 0.446 seconds


In [39]:
start = time.time()
test_data_summary_process2=[]
for i in range (0,length_test_data):
    if i%10000 == 0:
        print ("no.of rows reviews processed:",i)
    test_data_summary_process2.append(replacing_text(test_data1.summary[i]))
print("Training took {:0.3f} seconds".format(time.time()-start))

no.of rows reviews processed: 0
Training took 0.171 seconds


In [40]:
test_data1.data=pd.DataFrame(test_data_process2)
test_data1.summary=pd.DataFrame(test_data_summary_process2)

In [41]:
test_data1.head()

,summary,data,categories
0,order for mra,converted from care alert g...,PRESCRIPTION
1,patient sick wants rx,call from patient caller patient initial call ...,ASK_A_DOCTOR
2,fyi for doctor russo,call from patient caller patient initial call ...,ASK_A_DOCTOR
3,rx refills,call from other clinic initial call taken by a...,PRESCRIPTION
4,frequent falls refusing care call from wellspr...,other incoming call nursing supervisor at well...,ASK_A_DOCTOR


In [42]:
#Removal of Stop Words
stop = stopwords.words('english')
actual_data1['data'] = actual_data1['data'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
actual_data1['summary'] = actual_data1['summary'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

In [43]:
test_data1['data'] = test_data1['data'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
test_data1['summary'] = test_data1['summary'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

In [44]:
#Removal of user defined Stop Words
stop2 = ['phone','patient','please','back','caller','donna','shanon wright','cma','amy','daughter','taken','ann','thank','','initial','','x','r','call','a','m','mom','ravisankar','home','john','follow','details','detail','followup','pm','jones','erica','ma','md','called','russo','iii','http','add','also','like','ok', 'okay','pt','january','february','march','april','may','june','july','august','september','october','november','december']
actual_data1['data'] = actual_data1['data'].apply(lambda x: " ".join(x for x in x.split() if x not in stop2))
actual_data1['summary'] = actual_data1['summary'].apply(lambda x: " ".join(x for x in x.split() if x not in stop2))

In [45]:
#Removal of user defined Stop Words
test_data1['data'] = test_data1['data'].apply(lambda x: " ".join(x for x in x.split() if x not in stop2))
test_data1['summary'] = test_data1['summary'].apply(lambda x: " ".join(x for x in x.split() if x not in stop2))

In [46]:
freq_count_data = pd.Series(' '.join(actual_data1['data']).split()).value_counts()
freq_count_summary = pd.Series(' '.join(actual_data1['summary']).split()).value_counts()
print(freq_count_data,freq_count_summary)

mg              84910
clinical        66352
take            56499
doctor          54794
oral            52977
neurology       45535
relay           45032
tablets         40462
one             38432
rx              37857
tablet          37062
daily           36013
said            34562
notified        31261
day             30260
mouth           29052
ap              27249
rt              27014
weaver          26812
new             24754
need            24353
days            24034
get             22663
know            21966
shanon          21717
would           21687
wright          21578
pharmacy        21430
rogers          20906
see             20702
                ...  
myron               1
affilated           1
inprove             1
onther              1
autopsy             1
stephiane           1
thowing             1
inconnection        1
othropedic          1
equiment            1
summerstone         1
advi                1
levaquing           1
noifying            1
fiirst    

In [47]:
print(len(freq_count_data),len(freq_count_summary))
print(len(set(freq_count_data)),len(set(freq_count_summary)))

41868 12814
1435 339


In [68]:
from nltk.stem import PorterStemmer
from nltk.stem.lancaster import LancasterStemmer
st = PorterStemmer()
lt = LancasterStemmer()
actual_data1['data'] = actual_data1['data'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))
actual_data1['summary'] = actual_data1['summary'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))

In [69]:
test_data1['data'] = test_data1['data'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))
test_data1['summary'] = test_data1['summary'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))

In [70]:
actual_data1['data'] = actual_data1['data'].apply(lambda x: " ".join([lt.stem(word) for word in x.split()]))
actual_data1['summary'] = actual_data1['summary'].apply(lambda x: " ".join([lt.stem(word) for word in x.split()]))

In [71]:
test_data1['data'] = test_data1['data'].apply(lambda x: " ".join([lt.stem(word) for word in x.split()]))
test_data1['summary'] = test_data1['summary'].apply(lambda x: " ".join([lt.stem(word) for word in x.split()]))

In [72]:
actual_data1.data[0]

'bet c cough congest howev stil c low grad fev sleep oft say worr flu pcn allerg pharmac pick drug unclear nee abx bactrim on ds po bid gav paramet start list pcn allerg sulfamethoxazol trimethoprim mg or tablet sulfamethoxazol trimethoprim on po bid new allerg pcn penicillin v potass mod sulfamethoxazol trimethoprim mg or tablet sulfamethoxazol trimethoprim on po bid new allerg pcn penicillin v potass mod sulfamethoxazol trimethoprim mg or tablet sulfamethoxazol trimethoprim on po bid new allerg pcn penicillin v potass mod'

In [ ]:
###############################################################################################################################

In [118]:
actual_data1.to_csv("pre_processed_Guliford_actual_data2.csv",index=False)
test_data1.to_csv("pre_processed_Guliford_test_data2.csv",index=False)

In [8]:
#actual_data1=pd.read_csv("new_Guliford_data.csv",header=0)
#test_data1=pd.read_csv("new_Guliford_march_data.csv",header=0)

In [48]:
#doing TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_data_vectorizer = TfidfVectorizer(ngram_range=(1,),norm="l2",max_features= 5000)
tfidf_data = tfidf_data_vectorizer.fit_transform(actual_data1["data"])
dense_data = tfidf_data.todense()
tfidf_summary_vectorizer = TfidfVectorizer(ngram_range=(1,2),norm="l2",max_features= 4000)
tfidf_summary = tfidf_summary_vectorizer.fit_transform(actual_data1["summary"])
dense_summary = tfidf_summary.todense()
df_data = pd.DataFrame(dense_data)
df_summary = pd.DataFrame(dense_summary)

In [49]:
feature_names_data = tfidf_data_vectorizer.get_feature_names()
print(len(feature_names_data))
print(feature_names_data[:100])

5000
['ab', 'abd', 'abd pain', 'abdomen', 'abdominal', 'abdominal pain', 'able', 'able get', 'abnormal', 'abx', 'abx sent', 'acce', 'accident', 'according', 'accu', 'accu chek', 'acetaminophen', 'acetaminophen mg', 'acetaminophen one', 'acetaminophen take', 'acetate', 'acetonide', 'acetonide apply', 'acetonide ext', 'ache', 'aches', 'aches said', 'aching', 'achy', 'acid', 'acid reflux', 'across', 'act', 'act inh', 'act inhalation', 'act nasal', 'acting', 'action', 'action ap', 'action completed', 'action fwd', 'action information', 'action notified', 'action rx', 'action samples', 'active', 'activity', 'actually', 'acute', 'acyclovir', 'added', 'adderall', 'adderall mg', 'adderall xr', 'addition', 'additional', 'additional information', 'address', 'adjust', 'administer', 'admitted', 'advair', 'advanced', 'advanced care', 'advice', 'advil', 'advise', 'advise action', 'advise clinical', 'advise melissa', 'advised', 'advised action', 'advised ap', 'advised check', 'advised cna', 'advised 

In [50]:
feature_names_summary = tfidf_summary_vectorizer.get_feature_names()
print(len(feature_names_summary))
print(feature_names_summary[:5000])

4000
['ab', 'ab done', 'abcess', 'abd', 'abd pain', 'abdomen', 'abdomen pain', 'abdominal', 'abdominal discomfort', 'abdominal pain', 'abilify', 'able', 'abn', 'abnormal', 'abp', 'abp cuff', 'abx', 'abx answered', 'abx cough', 'abx dental', 'abx done', 'abx request', 'abx requested', 'abx sent', 'abx sinus', 'abx trip', 'abx uti', 'abxdone', 'acce', 'accident', 'ache', 'aches', 'aches fever', 'achiness', 'aching', 'achy', 'acid', 'acid reflux', 'acting', 'acute', 'acyclovir', 'added', 'adderal', 'adderall', 'adderall done', 'adderall refill', 'adderall refills', 'adderall rx', 'adderrall', 'adderrall done', 'additional', 'additional labs', 'adhd', 'adherance', 'adherance done', 'adherenace', 'adherence', 'adherence done', 'admission', 'admit', 'admitted', 'advair', 'advanced', 'advice', 'advise', 'advise regarding', 'advised', 'advised go', 'aetna', 'afford', 'afib', 'afternoon', 'agitation', 'ago', 'ahc', 'ahc calling', 'ahc needs', 'ahc rq', 'ahcdone', 'aid', 'aide', 'air', 'al', 'al

In [51]:
df_data = pd.DataFrame(dense_data)
df_summary = pd.DataFrame(dense_summary)
df_summary.shape

(86554, 4000)

In [52]:
tfidf_data_vectorizer = TfidfVectorizer(ngram_range=(1,2),norm="l2",max_features= 5000)
tfidf_test_data = tfidf_data_vectorizer.fit_transform(test_data1["data"])
dense_test_data = tfidf_test_data.todense()
tfidf_summary_vectorizer = TfidfVectorizer(ngram_range=(1,2),norm="l2",max_features= 4000)
tfidf_test_summary = tfidf_summary_vectorizer.fit_transform(test_data1["summary"])
dense_test_summary = tfidf_test_summary.todense()
df_test_data = pd.DataFrame(dense_test_data)
df_test_summary = pd.DataFrame(dense_test_summary)

In [53]:
df_test_data = pd.DataFrame(dense_test_data)
df_test_summary = pd.DataFrame(dense_test_summary)

In [54]:
preprocessed_test_data=pd.concat([df_test_data,df_test_summary],axis=1)
preprocessed_test_data.shape

(2279, 9000)

In [55]:
preprocessed_data=pd.concat([df_data,df_summary],axis=1)

In [56]:
preprocessed_data.shape

(86554, 9000)

In [57]:
preprocessed_data.columns

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            3990, 3991, 3992, 3993, 3994, 3995, 3996, 3997, 3998, 3999],
           dtype='int64', length=9000)

In [66]:
train_y=actual_data1.categories
TEST_Y=test_data1.categories

In [59]:
X_train_tfidf, X_test_tfidf, Y_train_tfidf, Y_test_tfidf = train_test_split(preprocessed_data, train_y, test_size=0.20, random_state=42)

In [60]:
import time
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier

In [61]:
start = time.time()
model_random = RandomForestClassifier(n_estimators = 1000,n_jobs = -1,random_state =1,max_features = "auto",max_depth=30 ,min_samples_leaf = 50)
model_fit = model_random.fit(X_train_tfidf,Y_train_tfidf)
print("Training took {:0.3f} seconds".format(time.time()-start))

Training took 386.258 seconds


In [62]:
pred_test = model_fit.predict(X_test_tfidf)
pred_test

array(['APPOINTMENTS', 'ASK_A_DOCTOR', 'APPOINTMENTS', ...,
       'PRESCRIPTION', 'ASK_A_DOCTOR', 'ASK_A_DOCTOR'], dtype=object)

In [63]:
print(accuracy_score(Y_test_tfidf,pred_test))

0.7225463578071746


In [64]:
pred_test_final = model_fit.predict(preprocessed_test_data)
pred_test_final

array(['ASK_A_DOCTOR', 'ASK_A_DOCTOR', 'ASK_A_DOCTOR', ...,
       'ASK_A_DOCTOR', 'ASK_A_DOCTOR', 'ASK_A_DOCTOR'], dtype=object)

In [67]:
print(accuracy_score(TEST_Y,pred_test_final))

0.419482229047828


In [ ]:
tokenizer_summary = Tokenizer(num_words=10000)
tokenizer_summary.fit_on_texts(actual_data1.summary)
train_summary_x = tokenizer_summary.texts_to_sequences(actual_data1.summary)
train_summary_x = pad_sequences(train_summary_x, maxlen=100)
train_summary_x=pd.DataFrame(train_summary_x)

In [ ]:
tokenizer_word=Tokenizer(num_words=10000)
tokenizer_word.fit_on_texts(actual_data1.data)
train_word_x=tokenizer_word.texts_to_sequences(actual_data1.data)
train_word_x=pad_sequences(train_word_x,maxlen=100)
train_word_x=pd.DataFrame(train_word_x)

final_train_x=pd.concat([train_summary_x,train_word_x],axis=1)

In [ ]:
tokenizer_summary.fit_on_texts(test_data1.summary)
tokenizer_word.fit_on_texts(test_data1.data)

test_summary_x= tokenizer_summary.texts_to_sequences(test_data1.summary)
test_data_x= tokenizer_word.texts_to_sequences(test_data1.data)

test_data_x=pad_sequences(test_data_x,maxlen=100)
test_summary_x=pad_sequences(test_summary_x,maxlen=100)


test_data_x= pd.DataFrame(test_data_x)
test_summary_x= pd.DataFrame(test_summary_x)

final_test_x=pd.concat([test_summary_x,test_data_x],axis=1)

In [ ]:
train_y=actual_data1.categories

In [ ]:
Test_Data_y=test_data1.categories

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(final_train_x, train_y, test_size=0.20, random_state=42)

In [ ]:
unique_labels = list(actual_data1.categories.unique())

train_y = np.array([unique_labels.index(i) for i in Y_train])
train_y = to_categorical(train_y)

test_y = np.array([unique_labels.index(i) for i in Y_test])
test_y = to_categorical(test_y)


Test_y = np.array([unique_labels.index(i) for i in Test_Data_y])
Test_y = to_categorical(Test_y)


In [ ]:
model = Sequential()
model.add(Embedding(100000,128))
model.add(LSTM(200,return_sequences=True,dropout=0.2,recurrent_dropout=0.2))
model.add(LSTM(200,return_sequences=True))
model.add(LSTM(200,return_sequences=True))
model.add(LSTM(200,return_sequences=False))
model.add(Dense(5))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

In [ ]:
batch_size=64
model.fit(X_train,
          train_y,
          batch_size=batch_size,verbose=2,epochs=1)

In [ ]:
test_predictions = model.predict_classes(X_test)

In [ ]:
list = []
#list1=[]
for i in test_predictions:
    print(unique_labels[i])
    list.append(unique_labels[i])

In [ ]:
pre=pd.DataFrame(list)

In [ ]:
accuracy_score(pre,Y_test)

In [ ]:
from nltk import ConfusionMatrix
from sklearn.metrics import accuracy_score,classification_report,precision_score,recall_score,precision_recall_curve,roc_curve,confusion_matrix



In [ ]:
tokenizer_test_summary = Tokenizer(num_words=6000)
tokenizer_test_summary.fit_on_texts(test_data1.summary)
test_summary_x = tokenizer_test_summary.texts_to_sequences(test_data1.summary)
test_summary_x = pad_sequences(test_summary_x, maxlen=50)
test_summary_x= pd.DataFrame(test_summary_x)

In [ ]:
tokenizer_test_word=Tokenizer(num_words=10000)
tokenizer_test_word.fit_on_texts(test_data1.data)
test_word_x=tokenizer_test_word.texts_to_sequences(test_data1.data)
test_word_x=pad_sequences(test_word_x,maxlen=50)
test_word_x=pd.DataFrame(test_word_x)

final_test_x=pd.concat([test_summary_x,test_word_x],axis=1)

In [ ]:
test_data_y=test_data1.categories

In [ ]:
test_data_predictions = model.predict_classes(final_test_x)
test_data_predictions

In [ ]:
#list1=[]
#for i in test_data_predictions:
#    print(unique_labels_test[i])
#    list.append(unique_labels_test[i])

In [ ]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

In [ ]:
model.save_weights("model2.h5")

In [ ]:
model.save('my_model.h5') 
print("Saved model to disk")


In [ ]:
json_file=open('model.json',"r")
loaded_model=json_file.read()
json_file.close()

In [ ]:
loaded_model1=model_from_json(loaded_model)

In [ ]:
loaded_model1.load_weights("model2.h5")

In [ ]:
# evaluate loaded model on test data
loaded_model1.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
#score = loaded_model1.evaluate(pre,Y_test, verbose=0)
#print("%s: %.2f%%" % (loaded_model1.metrics_names[1], score[1]*100))